In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.ops import box_convert
from source_code.card_detector import CardDetector, fit
from source_code.dataloader_utils import MTGCardsDataset, get_transform_pipe
from source_code.config import *

In [ ]:
anchor_boxes = torch.Tensor([[198.27963804, 206.74086672],
       [129.59395666, 161.90171490],
       [161.65437828, 232.34624509]
]) # Anchor boxes acquired from k-means clustering of the dataset

model = CardDetector(
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  anchor_boxes=anchor_boxes,
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
)

feature_map_dims = (model.features_w, model.features_h)

transform_pipe = get_transform_pipe(img_w=CONFIG["dataset"]["img_w"], img_h=CONFIG["dataset"]["img_h"])

train_dataset = MTGCardsDataset(
  annotations_file=CONFIG["dataset"]["annotations_file_train"],
  img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
  anchor_boxes=model.anchor_boxes,
  feature_map_dims=feature_map_dims,
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
  num_max_boxes=CONFIG["dataset"]["num_max_boxes"],
  transform=transform_pipe,
  limit=CONFIG["dataset"]["limit"]
)
train_dataloader = DataLoader(train_dataset, batch_size=CONFIG["dataloader"]["batch_size"])

val_dataset = MTGCardsDataset(
  annotations_file=CONFIG["dataset"]["annotations_file_val"],
  img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
  anchor_boxes=model.anchor_boxes,
  feature_map_dims=feature_map_dims,
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
  num_max_boxes=1,
  transform=transform_pipe,
  limit=CONFIG["dataset"]["limit"]
)
val_dataloader = DataLoader(val_dataset, batch_size=CONFIG["dataloader"]["batch_size"]) 

test_dataset = MTGCardsDataset(
  annotations_file=CONFIG["dataset"]["annotations_file_test"],
  img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
  anchor_boxes=model.anchor_boxes,
  feature_map_dims=feature_map_dims,
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
  num_max_boxes=CONFIG["dataset"]["num_max_boxes"],
  transform=transform_pipe,
  limit=CONFIG["dataset"]["limit"]
)
test_dataloader = DataLoader(test_dataset, batch_size=CONFIG["dataloader"]["batch_size"]) 

In [ ]:
opt = torch.optim.Adam(params=model.parameters(), lr=CONFIG["optimizer"]["lr"])
fit(
    model=model,
    num_epochs=CONFIG["optimizer"]["num_epochs"],
    optimizer=opt, 
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=model.device
)

In [ ]:
import torchvision.transforms.functional as fn
from torchvision.io import read_image, ImageReadMode
from torchvision.utils import draw_bounding_boxes


images, labels = next(iter(test_dataloader))

# Load the unaltered (not normalized) version of the image
test_image = read_image("./data/aug_test/10621_1.png", mode=ImageReadMode.RGB)
index = 20

# Predict the bounding box and get the true box
model = model.to(device=model.device)

image = images[index]
pred_boxes = model.predict(image, keep_box_score_treshhold=0, num_max_boxes=1)
pred_box = pred_boxes.squeeze(0)

true_boxes = model.predict(image, ground_truth=labels[index].unsqueeze(0))
true_box = true_boxes.squeeze(0)[:1]

print("pred:", pred_box)
print("true:",true_box)

bbox_img_tensor = draw_bounding_boxes(image=test_image, boxes=pred_box, width=3, colors=(255,255,0))

box_conv = box_convert(pred_box, in_fmt='xyxy', out_fmt='cxcywh')
location_cell = torch.Tensor([[box_conv[0], box_conv[1], box_conv[0]+32, box_conv[1]+32]])
bbox_img_tensor = draw_bounding_boxes(image=test_image, boxes=location_cell, width=3, colors=(255,255,0))


fn.to_pil_image(bbox_img_tensor)

In [ ]:
pred_box

In [ ]:
true_box

In [ ]:
bbox_img_tensor = draw_bounding_boxes(image=test_image, boxes=true_box, width=3, colors=(255,255,0))

fn.to_pil_image(bbox_img_tensor)

In [ ]:
fn.to_pil_image(image)

In [ ]:
import numpy as np

position = [175.5,354.0,107,256]

anchors_array = np.array(anchor_boxes)
anchor_boxes_to_draw = torch.Tensor([[position[0] - anchor[0]/2, position[1] - anchor[1]/2, position[0] + anchor[0]/2, position[1] + anchor[1]/2] for anchor in anchors_array])
bbox_img_tensor = draw_bounding_boxes(image=test_image, boxes=anchor_boxes_to_draw, width=3, colors=(150,150,150))

box = [position[0] - position[2]/2, position[1] - position[3]/2, position[0] + position[2]/2, position[1] + position[3]/2]
boxes = torch.Tensor([box])
bbox_img_tensor = draw_bounding_boxes(image=bbox_img_tensor, boxes=boxes, width=3, colors=(255,255,0))

fn.to_pil_image(bbox_img_tensor)